In [1]:
import numpy as np
import shorttext

/Users/stephenhky/anaconda/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# loading the whole ESV Bible
import biblebooks.bibledocs_iterator as bit
dbconn = bit.get_sqlite3_dbconn("ESVCrossway.sqlite")
biblecorpus = bit.generate_classdict_chapters(dbconn)

In [3]:
from biblebooks import BibleAbbrDict

In [4]:
# loading FastText model
wvmodel = shorttext.utils.load_fasttext_model('/Users/stephenhky/Data/FastText/wiki.simple/wiki.simple.bin')

In [5]:
# define neural network

In [6]:
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, LSTM, Activation, Input
from keras.models import Sequential, Model
from keras.regularizers import l2

In [7]:
maxlen = 20

input_x = Input(shape=(maxlen, wvmodel.vector_size))
output_y = LSTM(1024, return_sequences=True)(input_x)
print(output_y.shape)
output_y = Conv1D(filters=1200, kernel_size=2)(output_y)
print(output_y.shape)
output_y = MaxPooling1D(pool_size=19)(output_y)
print(output_y.shape)
output_y = Flatten()(output_y)
print(output_y.shape)
output_y = Dense(2, activation='sigmoid')(output_y)
print(output_y.shape)

kmodel = Model(inputs=input_x, outputs=output_y)

(?, ?, 1024)
(?, ?, 1200)
(?, ?, 1200)
(?, ?)
(?, 2)


In [8]:
kmodel.compile(loss='categorical_crossentropy', optimizer='adam')

In [9]:
maxlen = 20

smodel = Sequential()
smodel.add(LSTM(1024, return_sequences=True, input_shape=(maxlen, wvmodel.vector_size)))
smodel.add(Conv1D(filters=1200, kernel_size=2))
smodel.add(MaxPooling1D(pool_size=maxlen-2+1))
smodel.add(Flatten())
smodel.add(Dense(2, activation='sigmoid'))

smodel.compile(loss='categorical_crossentropy', optimizer='adam')

In [10]:
newdict = {}
newdict['ot'] = [sent for otbookabbr in BibleAbbrDict.otbookdict  if otbookabbr=='de'
                 for chap in range(1, BibleAbbrDict.numchaps[otbookabbr]+1) 
                 for sent in biblecorpus[otbookabbr+'_'+str(chap)]]
newdict['nt'] = [sent for ntbookabbr in BibleAbbrDict.ntbookdict if ntbookabbr=='mt'
                 for chap in range(1, BibleAbbrDict.numchaps[otbookabbr]+1) 
                 for sent in biblecorpus[otbookabbr+'_'+str(chap)]]

In [11]:
classifier = shorttext.classifiers.VarNNEmbeddedVecClassifier(wvmodel, maxlen=maxlen)

In [12]:
classifier.train(newdict, kmodel)

Epoch 1/10
1297/1297 [==============================] - 58s 45ms/step - loss: 0.5275

In [13]:
classifier.score('Love the Lord with all your heart.')

{'nt': 0.22094691, 'ot': 0.9791533}

In [14]:
classifier.score('Pray in this way as I have instructed you.')

{'nt': 0.23150909, 'ot': 0.9828587}

In [18]:
classifier.score('Moses speaks.')

{'nt': 0.23335962, 'ot': 0.9814356}

In [16]:
classifier.score('Jesus Christ')

{'nt': 0.22772633, 'ot': 0.9858623}

In [17]:
classifier.score('False idols and true son of God.')

{'nt': 0.21462741, 'ot': 0.98284227}

In [19]:
classifier.score('Abraham died.')

{'nt': 0.24620603, 'ot': 0.96869195}

In [20]:
classifier.save_compact_model('de_mt_model.bin')